# Tallying corner parking violations

The `parking_violation` table has two columns (`street_name` and `intersecting_street`) with New York City streets. When the values for both columns are not `NULL`, this indicates that the violation occurred on a corner where two streets intersect. In an effort to identify street corners that tend to be the location of frequent parking violations, you have been tasked with identifying which violations occurred on a street corner and the total number of violations at each corner.

In this exercise, you will concatenate the `street_name` and `intersecting_street` columns to create a new `corner` column. Then all parking violations occurring at a corner will be tallied by a SQL query.

```
SELECT
  -- Combine street_name, ' & ', and intersecting_street
  street_name || ' & ' || intersecting_street AS corner
FROM
  parking_violation;
```

```
SELECT
  -- include the corner in results
  corner,
  -- include the total number of violations occurring at corner
  COUNT(corner)
FROM (
  SELECT
    -- concatenate street_name, ' & ', and intersecting_street
    street_name || ' & ' || intersecting_street AS corner
  FROM
    parking_violation
) sub
WHERE
  -- exclude corner values that are NULL
  corner IS NOT NULL
GROUP BY
  corner
ORDER BY
  count DESC
```

# Creating a TIMESTAMP with concatenation

In a previous exercise, the `violation_time` column in the `parking_violation` table was used to check that the recorded `violation_time` is within the violation location's restricted times. This presented a challenge in cases where restricted parking took place overnight because, for these records, the `from_hours_in_effect` time is later than the `to_hours_in_effect` time. This issue could be eliminated by including a date in addition to the time of a violation.

In this exercise, you will begin the process of simplifying the identification of overnight violations through the creation of the `violation_datetime` column populated with `TIMESTAMP` values. This will be accomplished by concatenating `issue_date` and `violation_time` and converting the resulting strings to `TIMESTAMP` values.

```
SELECT
  -- Convert violation_time to TIMESTAMP
  TO_TIMESTAMP(violation_datetime, 'MM/DD/YYYY HH12MIAM') AS violation_datetime
FROM (
  SELECT
    CONCAT(issue_date, ' ', violation_time) AS violation_datetime
  FROM
    parking_violation
) sub;
```

# Extracting time units with SUBSTRING()

In a previous exercise, you separated the interval between the `violation_time` and `to_hours_in_effect` columns into their constituent `hour` and `minute` time units. Some pre-cleaning of these values was done behind the scenes to make the values more amenable for conversion because of inconsistencies in the recording of these values. The functions explored in this lesson provide an approach to extract values from strings.

In this exercise, you will use `SUBSTRING()` to extract the hour and minute units from time strings. This is an alternative approach to extracting time units removing the need to convert the string to a `TIMESTAMP` value to extract the time unit as was done previously.

```
SELECT
  SUBSTRING(violation_time FROM 1 FOR 2) AS hour,
  -- Define minute column
  SUBSTRING(violation_time FROM 3 FOR 2) AS minute
FROM
  parking_violation;
```

# Extracting house numbers from a string

Addresses for the Queens borough of New York City are unique in that they often include dashes in the house number component of the street address. For example, for the address `86-16 60 Ave`, the house number is `16`, and `86` refers to the closest cross street. Therefore, if we want the house_number to strictly represent the house number where a parking violation occurred, we need to extract the digits after the dash (-) to represent this value.

In this exercise, you will use `STRPOS()`, `SUBSTRING()`, and `LENGTH()` to extract the specific house number from Queens street addresses.

```
SELECT
  house_number,
  -- Extract the substring after '-'
  SUBSTRING(
    -- Specify the column of the original house number
    house_number
    -- Calculate the position that is 1 beyond '-'
    FROM STRPOS(house_number, '-') + 1
    -- Calculate number characters from dash to end of string
    FOR LENGTH(house_number) - STRPOS(house_number, '-')
  ) AS new_house_number
FROM
  parking_violation;
```

# Splitting house numbers with a delimiter

In the previous exercise, you used `STRPOS()`, `LENGTH()`, and `SUBSTRING()` to separate the actual house number for Queens addresses from the value representing a cross street. In the video exercise, you learned how strings can be split into parts based on a delimiter string value.

In this exercise, you will extract the house number for Queens addresses using the `SPLIT_PART()` function.

```
SELECT
  -- Split house_number using '-' as the delimiter
  SPLIT_PART(house_number, '-', 2) AS new_house_number
FROM
  parking_violation
WHERE
  violation_county = 'Q';
```

# Mapping parking restrictions

You are interested in building a mobile parking recommendation app for New York City. The goal is to use the `parking_violation` dataset to map parking restrictions to a driver's location. Parking restrictions are stored in the `days_parking_in_effect` column in a format that consists of a string of 7 characters. Each position in the string represents a day of the week (Monday-Sunday). A B indicates parking is restricted and a Y indicates parking is allowed. A colleague has organized the data from `parking_violation` by creating a table named `parking_restrictions`, which includes the `street_address`, `violation_county`, and `days_parking_in_effect`.

In this exercise, you will use `REGEXP_SPLIT_TO_TABLE()` and `ROW_NUMBER()` to associate each street address to its parking availability.

```
SELECT
  -- Specify SELECT list columns
  street_address,
  violation_county,
  REGEXP_SPLIT_TO_TABLE(days_parking_in_effect, '') AS daily_parking_restriction
FROM
  parking_restriction;
```

```
SELECT
  -- Label daily parking restrictions for locations by day
  ROW_NUMBER() OVER(
    PARTITION BY
        street_address, violation_county
    ORDER BY
        street_address, violation_county
  ) AS day_number,
  *
FROM (
  SELECT
    street_address,
    violation_county,
    REGEXP_SPLIT_TO_TABLE(days_parking_in_effect, '') AS daily_parking_restriction
  FROM
    parking_restriction
) sub;
```

# Selecting data for a pivot table

In an effort to get a better understanding of which agencies are responsible for different types of parking violations, you have been tasked with creating a report providing these details. This report will focus on four issuing agencies: `Police Department` (`P`), `Department of Sanitation` (`S`), `Parks Department` (`K`), and `Department of Transportation` (`V`). All of the records required to create such a report are present in the `parking_violations` table. An `INTEGER` `violation_code` and `CHAR` `issuing_agency` is recorded for every `parking_violation`.

In this exercise, you will write a `SELECT` query that provides the underlying data for your report: the parking violation code, the issuing agency code, and the total number of records with each pair of values.

```
SELECT 
	-- Include the violation code in results
	violation_code, 
    -- Include the issuing agency in results
    issuing_agency, 
    -- Number of records with violation code/issuing agency
    COUNT(*) 
FROM 
	parking_violation 
WHERE 
	-- Restrict the results to the agencies of interest
	issuing_agency IN ('P', 'S', 'K', 'V') 
GROUP BY 
	-- Define GROUP BY columns to ensure correct pair count
	violation_code, issuing_agency
ORDER BY 
	violation_code, issuing_agency;
```

# Using FILTER to create a pivot table

In the previous exercise, you wrote a query that provided information on the number of parking violations (by their numerical code) issued by each of four agencies. The results contained all of the desired information but were presented in a format that included a duplicate display of each `violation_code` up to four times (for every `issuing_agency` selected) in the results. A more compact representation of the same data can be achieved through the creation of a pivot table.

In this exercise, you will write a query using the `FILTER` clause to produce results in a pivot table format. This improved presentation of the data can more easily be used in the report for parking violations issued by each of the four agencies of interest.

```
SELECT 
	violation_code, 
    -- Define the "Police" column
	COUNT(issuing_agency) FILTER (WHERE issuing_agency = 'P') AS "Police",
    -- Define the "Sanitation" column
	COUNT(issuing_agency) FILTER (WHERE issuing_agency = 'S') AS "Sanitation",
    -- Define the "Parks" column
	COUNT(issuing_agency) FILTER (WHERE issuing_agency = 'K') AS "Parks",
    -- Define the "Transportation" column
	COUNT(issuing_agency) FILTER (WHERE issuing_agency = 'V') AS "Transportation"
FROM 
	parking_violation 
GROUP BY 
	violation_code
ORDER BY 
	violation_code
```

# Aggregating film categories

For the final exercise in this course, let's return to the `film_permit` table. It contains a `community_board` `TEXT` column composed of a comma-separated list of integers. There is interest in doing an analysis of the types of film permits that are being provided for each community board. However, the representation of community boards (`INTEGER`s in a `TEXT` column) makes this difficult. By using techniques learned in this chapter, the data can be transformed to allow for such an analysis.

In this exercise, you will first create a (temporary) `VIEW` that represents the `community_board` values individually for two permit categories. A `VIEW` is a named query that can be used like a `TABLE` once created. You will use this `VIEW` in a subquery for aggregating the results in a pivot table.

```
CREATE OR REPLACE TEMP VIEW cb_categories AS 
SELECT
	-- Split community board values
	REGEXP_SPLIT_TO_TABLE(community_board, ', ') AS community_board,
    category
FROM
	film_permit
WHERE 
    -- Restrict the categories in results
    category IN ('Film', 'Television', 'Documentary');

-- View cb_categories
SELECT * FROM cb_categories;
```

```
SELECT
	-- Convert community_board data type
	community_board::INT AS community_board,
    -- Define pivot table columns
	COUNT(category) FILTER(where category = 'Film') AS "Film",
    COUNT(category) FILTER(where category = 'Television') AS "Television",
	COUNT(category) FILTER(where category = 'Documentary') AS "Documentary"
FROM
	cb_categories
GROUP BY 
	community_board
ORDER BY 
	community_board;
```